## Gerekli kütüphaneler:
Modelin kurulumu ve eğitim süreci için gerekli tüm paketleri içe aktarılır.

In [3]:
import sys

sys.path.append("../..")

import os
import cv2
import numpy as np
import joblib
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from utils.ml_features.preprocessing import extract_features, apply_pca
from utils.ml_features.grid_search import grid_search

## Veri yolu ve etiketler:
Veri setinin konumu `data_dir` değişkeninde tanımlanır ve klasör isimleri alfabetik olarak sıralanarak `class_names` listesi ile sınıf etiketleri elde edilir.

In [4]:
data_dir = "../../data/skyview"
class_names = sorted(os.listdir(data_dir))

## Feature çıkarma:
Her sınıfa ait tüm görüntüler okunur, 128×128 boyutuna getirilir ve RGB formatına dönüştürülür. `extract_features` fonksiyonu ile HOG, LBP, HSV histogramı ve GLCM tabanlı özellikler çıkarılarak `x_data` dizisine eklenir. Sınıf etiketleri `y_data` listesinde tutulur.

In [5]:
x_data, y_data = [], []

for label, cls in enumerate(class_names):
    cls_path = os.path.join(data_dir, cls)
    print("İşlenen class:", cls)
    for fname in tqdm(os.listdir(cls_path), desc=f"{cls}", leave=False):
        img_path = os.path.join(cls_path, fname)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        features = extract_features(img)
        x_data.append(features)
        y_data.append(label)

x_data = np.array(x_data)
y_data = np.array(y_data)
print("Toplam örnek:", len(x_data), "| Feature boyutu:",  x_data.shape[1])

İşlenen class: Agriculture


İşlenen class: Airport


İşlenen class: Beach


İşlenen class: City


İşlenen class: Desert


...

İşlenen class: Port


İşlenen class: Railway


İşlenen class: Residential


İşlenen class: River


Toplam örnek: 12000 | Feature boyutu: 1803


## PCA kullanarak boyut azaltımı:
`apply_pca` fonksiyonu ile 100 bileşenli PCA uygulanarak özellik vektörlerinin boyutu küçültülür.

In [6]:
x_reduced = apply_pca(True, x_data, n_components=100)

## Veriyi bölme ve kaydetme:
PCA sonrası özellikler, %80 eğitim ve %20 test olacak şekilde `stratify` ile sınıf dengesi korunarak ayrılır. Elde edilen `x_train`, `y_train`, `x_test`, `y_test` verileri `.npy` formatında kaydedilir.

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y_data, test_size=0.2, stratify=y_data, random_state=42)

np.save("../../features/skyview_x_train.npy", x_train)
np.save("../../features/skyview_y_train.npy", y_train)
np.save("../../features/skyview_x_test.npy", x_test)
np.save("../../features/skyview_y_test.npy", y_test)

## Grid search ile random forest:
Random Forest modeli için ağaç sayısı `n_estimators` ve maksimum derinlik `max_depth`  
hiperparametreleri 5 katlı çapraz doğrulama ile taranır. En iyi model kaydedilerek daha sonra test ve analiz aşamalarında kullanılır.`skyview_rf.pkl`

In [7]:
model = RandomForestClassifier()
params = {
    "n_estimators": [75, 100],
    "max_depth": [10, 20]
}

print("Random Forest eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/skyview_rf.pkl")
print("Random Forest için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Random Forest eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 4


RandomForestClassifier Grid Search:  25%|██▌       | 1/4 [00:16<00:48, 16.27s/it]

Parametreler: {'n_estimators': 75, 'max_depth': 10} | Skor: 0.6759375 | Süre: 16 sn


RandomForestClassifier Grid Search:  50%|█████     | 2/4 [00:35<00:35, 17.84s/it]

Parametreler: {'n_estimators': 75, 'max_depth': 20} | Skor: 0.7202083333333332 | Süre: 18 sn


RandomForestClassifier Grid Search:  75%|███████▌  | 3/4 [00:49<00:16, 16.43s/it]

Parametreler: {'n_estimators': 100, 'max_depth': 10} | Skor: 0.6816666666666666 | Süre: 14 sn


RandomForestClassifier Grid Search: 100%|██████████| 4/4 [01:11<00:00, 17.95s/it]

Parametreler: {'n_estimators': 100, 'max_depth': 20} | Skor: 0.7259374999999999 | Süre: 21 sn


Random Forest için en iyi skor: 0.7259374999999999 | Parametreler: {'n_estimators': 100, 'max_depth': 20} | Toplam Süre: 1 dk 11 sn 



## Grid search ile gradient boosting:
Gradient Boosting için `n_estimators` hiperparametresi 5 katlı çapraz doğrulama ile taranır. En iyi ortalama skoru veren model seçilip kaydedilir. `skyview_gb.pkl`

In [8]:
model = GradientBoostingClassifier()
params = {
    "n_estimators": [100, 200]
}

print("Gradient Boosting eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/skyview_gb.pkl")
print("Gradient Boosting için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Gradient Boosting eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 2


GradientBoostingClassifier Grid Search:  50%|█████     | 1/2 [16:21<16:21, 981.76s/it]

Parametreler: {'n_estimators': 100} | Skor: 0.7387499999999999 | Süre: 16 dk 21 sn


GradientBoostingClassifier Grid Search: 100%|██████████| 2/2 [49:16<00:00, 1478.39s/it]

Parametreler: {'n_estimators': 200} | Skor: 0.7584375000000001 | Süre: 32 dk 55 sn


Gradient Boosting için en iyi skor: 0.7584375000000001 | Parametreler: {'n_estimators': 200} | Toplam Süre: 49 dk 16 sn 



## Grid search ile multilayer perceptron:
MLP için katman yapısı `hidden_layer_sizes`, düzenleme katsayısı `alpha` ve  
öğrenme oranı `learning_rate_init` hiperparametreleri 5 katlı çapraz doğrulama ile taranır. En iyi skor elde eden model kaydedilir. `skyview_mlp.pkl`

In [9]:
model = MLPClassifier(
    max_iter=1500,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=20,
    tol=1e-5
)
params = {
    "hidden_layer_sizes": [(256,), (256, 128)],
    "alpha": [0.001, 0.005],
    "learning_rate_init": [0.001]
}

print("Multilayer Perceptron eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/skyview_mlp.pkl")
print("Multilayer Perceptron için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Multilayer Perceptron eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 4


MLPClassifier Grid Search:  25%|██▌       | 1/4 [00:24<01:12, 24.06s/it]

Parametreler: {'hidden_layer_sizes': (256,), 'alpha': 0.001, 'learning_rate_init': 0.001} | Skor: 0.5919791666666667 | Süre: 24 sn


MLPClassifier Grid Search:  50%|█████     | 2/4 [00:47<00:47, 23.96s/it]

Parametreler: {'hidden_layer_sizes': (256,), 'alpha': 0.005, 'learning_rate_init': 0.001} | Skor: 0.5865625 | Süre: 23 sn


MLPClassifier Grid Search:  75%|███████▌  | 3/4 [01:28<00:31, 31.50s/it]

Parametreler: {'hidden_layer_sizes': (256, 128), 'alpha': 0.001, 'learning_rate_init': 0.001} | Skor: 0.58375 | Süre: 40 sn


MLPClassifier Grid Search: 100%|██████████| 4/4 [02:12<00:00, 33.11s/it]

Parametreler: {'hidden_layer_sizes': (256, 128), 'alpha': 0.005, 'learning_rate_init': 0.001} | Skor: 0.5712499999999999 | Süre: 44 sn


Multilayer Perceptron için en iyi skor: 0.5919791666666667 | Parametreler: {'hidden_layer_sizes': (256,), 'alpha': 0.001, 'learning_rate_init': 0.001} | Toplam Süre: 2 dk 12 sn 

